In [33]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

In [4]:
data = pd.read_csv('./data/nba_rookies.csv')
data.head(2)

,Name,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,...,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,No
1,Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,...,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,No


In [7]:
data.columns = data.columns.str.lower()

In [110]:
data.describe()

,gp,min,pts,fgm,fga,fg%,3p made,3pa,3p%,ftm,fta,ft%,oreb,dreb,reb,ast,stl,blk,tov,target_5yrs
count,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000
mean,60.402558,17.647931,6.820166,2.634763,5.902483,44.117532,0.249661,0.785628,19.308126,1.302784,1.827314,70.407148,1.007600,2.021445,3.028442,1.558992,0.620391,0.366140,1.196012,0.621520
std,17.481092,8.327441,4.365671,1.686850,3.598820,6.119492,0.384608,1.063856,16.022916,0.989341,1.326655,10.489838,0.779178,1.362464,2.062357,1.474159,0.410533,0.428534,0.724470,0.485191
min,11.000000,3.100000,0.700000,0.300000,0.800000,23.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.300000,0.000000,0.000000,0.000000,0.100000,0.000000
25%,47.000000,10.800000,3.700000,1.400000,3.300000,40.200000,0.000000,0.000000,0.000000,0.600000,0.900000,64.800000,0.400000,1.000000,1.500000,0.600000,0.300000,0.100000,0.700000,0.000000
50%,63.000000,16.100000,5.600000,2.100000,4.800000,43.900000,0.100000,0.300000,22.400000,1.000000,1.500000,71.300000,0.800000,1.700000,2.500000,1.100000,0.500000,0.200000,1.000000,1.000000
75%,77.000000,22.900000,8.800000,3.400000,7.500000,47.800000,0.400000,1.200000,32.500000,1.700000,2.300000,77.600000,1.400000,2.600000,4.000000,2.000000,0.800000,0.500000,1.500000,1.000000
max,82.000000,40.900000,28.200000,10.200000,19.800000,73.700000,2.300000,6.500000,100.000000,7.700000,10.200000,100.000000,5.300000,9.600000,13.900000,10.600000,2.500000,3.900000,4.400000,1.000000


In [8]:
data.columns

Index(['name', 'gp', 'min', 'pts', 'fgm', 'fga', 'fg%', '3p made', '3pa',
       '3p%', 'ftm', 'fta', 'ft%', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk',
       'tov', 'target_5yrs'],
      dtype='object')

In [9]:
data.index = data['name']

In [11]:
data.drop(columns=['name'], inplace=True)

In [12]:
data.head(2)

,gp,min,pts,fgm,fga,fg%,3p made,3pa,3p%,ftm,fta,ft%,oreb,dreb,reb,ast,stl,blk,tov,target_5yrs
name,,,,,,,,,,,,,,,,,,,,
Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,No
Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,No


In [16]:
data['target_5yrs'] = data['target_5yrs'].map({'No':0, 'Yes':1})
data.head(7)


,gp,min,pts,fgm,fga,fg%,3p made,3pa,3p%,ftm,fta,ft%,oreb,dreb,reb,ast,stl,blk,tov,target_5yrs
name,,,,,,,,,,,,,,,,,,,,
Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0
Andrew Harrison,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0
JaKarr Sampson,74,15.3,5.2,2.0,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67.0,0.5,1.7,2.2,1.0,0.5,0.3,1.0,0
Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1.0,0.9,1.9,0.8,0.6,0.1,1.0,1
Matt Geiger,48,11.5,4.5,1.6,3.0,52.4,0.0,0.1,0.0,1.3,1.9,67.4,1.0,1.5,2.5,0.3,0.3,0.4,0.8,1
Tony Bennett,75,11.4,3.7,1.5,3.5,42.3,0.3,1.1,32.5,0.4,0.5,73.2,0.2,0.7,0.8,1.8,0.4,0.0,0.7,0
Don MacLean,62,10.9,6.6,2.5,5.8,43.5,0.0,0.1,50.0,1.5,1.8,81.1,0.5,1.4,2.0,0.6,0.2,0.1,0.7,1


In [101]:
knn = KNeighborsClassifier(25)
ss = StandardScaler()

X = data.drop(columns=['target_5yrs'])
y = data['target_5yrs']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [102]:
y_train.value_counts(normalize=True) # baseline accuracy score: 62%

1    0.621486
0    0.378514
Name: target_5yrs, dtype: float64

In [103]:
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [104]:
y.value_counts(normalize=True)

1    0.62152
0    0.37848
Name: target_5yrs, dtype: float64

In [105]:
y_test.value_counts(normalize=True)

1    0.621622
0    0.378378
Name: target_5yrs, dtype: float64

In [106]:
cross_val_score(knn, X_train_sc, y_train, cv=10).mean()

0.6808181818181818

In [107]:
knn.fit(X_train_sc, y_train)

KNeighborsClassifier(n_neighbors=25)

In [108]:
knn.score(X_train_sc, y_train)

0.713855421686747

In [109]:
knn.score(X_test_sc, y_test) # ~9% above baseline

0.7087087087087087

In [111]:
y_preds = pd.DataFrame(knn.predict(X_test_sc), columns=['predictions'], index=y_test.index)
y_preds

,predictions
name,
Jerrod Mustaf,1
Everette Stephens,0
Isiah Thomas*,1
Sean Higgins,1
Kris Humphries,1
...,...
Keith Lee,1
Marlon Maxey,1
Chris Singleton,0


In [112]:
y_preds.to_csv('./data/predictions.csv')